Na dobry początek napiszemy funkcję która będzie szacowała liczbę sukcesów w ciągu prób Bernoulliego na podstawie twierdzenia Poissona. Dla przypomnienia twiedzenie to mówi, że jeżeli mamy ciągi $n$, $p_n$ i $np_n$ z granicami $n \rightarrow \infty$, $p_n \rightarrow 0$ i $np_n \rightarrow \lambda$ to $$ \binom{n}{k}p_n^k(1-p_n)^{n-k} \rightarrow_{n\rightarrow \infty} \frac{\lambda^k}{k!}e^{-\lambda} .$$ Innymi słowy jeżeli mamy odpowiednio duże $n$ przy stosunkowo małym $p$ to wyliczenie prawdopodobieństwa uzyskania dokładnie $k$ sukcesów w $n$ niezależnych próbach Bernoulliego można zastąpić wyliczeniem prawdopodobienstwem otrzymania wartości $k$ w rozkładzie Poissona z parametrem $\lambda = np$. Zatem nasza pierwsza funkcja ma za zadanie przyjąć oczekiwaną liczbę prób, prawdopodobieństwo sukcesu, a także szukaną liczbę sukcesów i wyznaczyć odpowiednią wartość z rozkładu Poissona.

In [2]:
from scipy.stats import norm
from math import exp, sqrt, factorial, ceil
inf = float('inf')

def poisson(n, p, k):
  #n liczba prób; p prawdopodobieństwo sukcesu; k liczba szukanych sukcesów
  lambd = n*p
  expression = lambd**k/(factorial(k)*exp(lambd))
  return expression

#przykład zastosowania poisson(10000000,0.00000007,1) daje wynik 0.34760971265398666

0.34760971265398666

Twierdzenie Poissona miało również wersję która pozwalała nam określić dokładność naszego oszacowania, nie tylko dla jednej konkretnej liczby sukcesów, ale dla dowolnego podzbioru lliczby możliwych sukcesów. Dla przypomnienia zakładaliśmy, że zmienna losowa $S_n$ posiada rozkład Bernoulliego z parametrami $n$ i $p$, oraz, że $\lambda =np$. Wtedy dla dowolnego zbioru $B \subset \{ 0,1,2,...\} $ mamy $$ {\huge |} P(S_n \in B) - \sum_{k \in B} \frac{\lambda^k}{k!} e^{-\lambda} {\huge |} \leq \frac{\lambda^2}{n} .$$ Praktycznie zatem mamy oszacowanie na to na ile dokłady jest nasz wynik przy zadanych parametrach. Następna funkcja jest dodatkowa i ma za zadanie wykożystać właśnie to twierdzenie. Na wejściu mamy podać parametry rozkładu $n$, $p$, nie pojedynczą wartość liczby sukcesów ale (potencjalnie) całą listę interesujących nas wartości i następnie wyznaczyć szacowanie przez rozkład Poissona prawopodobieństwa otrzymania liczby sukcesów z tej listy, oraz dokładność naszego przybliżenia (czyli $\lambda^2/n$).

In [3]:
def poisson_ext(n,p,k):#dod
  #n liczba prób; p prawdopodobieństwo sukcesu; k lista ilości szukanych sukcesów
  lambd=n*p
  expressions=[lambd**i/(factorial(i)*exp(lambd)) for i in k]
  expression=sum(expressions)
  return expression, (lambd**2/n)#para: prawdopodobieństwo otrzymania liczby sukcesów podanej w liście oraz błąd oszacowania
#przykład zastosowania poisson_ext(10000000,0.00000007,[0,1,2]) daje parę [0.9658584158742916, 4.900000000000001e-08]

(0.9658584158742916, 4.900000000000001e-08)

Kolejną funkcją którą mamy stworzyć oparta jest o Centralne Twierdzenie Graniczne. Niech $,X_1,X_2,...$ będą niezależnymi zmiennymi losowymi o tym samym rozkładzie, wartości oczekiwanej $m$ i wariancji $\sigma^2 >0$. Wtedy dla każdego $t$ $$ P(\frac{X_1+...+X_n -nm}{\sigma \sqrt{n}} \leq t) \rightarrow_{n \rightarrow \infty} \Phi (t) ,$$ gdzie $\Phi (t)$ jest dystrybuantą rozkładu normalnego o wartości oczekiwanej 0 i wariancji 1. Innymi słowy jeżeli chcemy, oczacować prawdopodobieństwo z jakim suma naszych niezależnych zmiennych losowych trafi do pewnego przedziału $[a,b]$ ($a$ i $b$ mogą w szczególności przyjąć wartości odpowiednio minus i plus nieskończoność), to wystarczy nam policzyć dystrybuantę rozkładu normalnego $\mathcal{N} (0,1)$ w punktach $\frac{b-nm}{\sigma\sqrt{n}}$ i $\frac{a-nm}{\sigma \sqrt{n}} $ i te wartości od siebie odjąć(w celu liczenia wartości dystrybuanty w danym punkcie używamy funkcji $norm.cdf$). Tym samym do funkcji mamy podać liczbę zmiennych $n$, ich wartość oczekiwaną $m$, odchylenie standardowe $s$ (które jest pierwiastkiem z wariancji), oraz ograniczenia przediału który nas interesuje.    

In [4]:
def CTG(n,m,s,d=-inf ,g=inf):
  #n liczba zmiennych, m- wartość oczekiwana zmiennych, s- odchylenie standardowe zmiennych, d -dolne ograniczenie, g- górne ograniczenie
  b=(g-n*m)/(s*sqrt(n))
  a=(d-n*m)/(s*sqrt(n))
  return norm.cdf(b)-norm.cdf(a) #szacowane prawdopodobieństwo tego że suma podanych n niezależnych zmiennych losowych wpadnie w przedział od d do g na podstawie centralnego twierdzenia granicznego
#przykład zastosowania CTG(400,0.3,sqrt(0.3*0.7), d=130) daje wynik 0.13761676203741713

np.float64(0.13761676203741713)

Z CTG również możemy korzystać na inne sposoby, na przykład na podstawie posiadanych informacji oszacować jaka jest minimalna liczba elementów konieczna by uzyskać daną dokładność z odpowiednim prawdopodobieństwem. Rozważmy na przykład schemat Bernoulliego. Wiadomo, że oczekiwaną wartością sukcesów w $n$ próbach jest $np$ gdzie $p$ to prawdopodobieństwo sukcesu w pojedyńczej próbie. Jeżeli chcemy znaleźć prawdopodobieństwo że odchylimy się nie więcej niż o ustaloną wartość $d$ od naszej wartości średniej musimy znaleźć $$ P( np-d \leq S_n \leq np+d) $$, gdzie $S_n$ to liczba sukcesów w naszym schemacie. Oczywiście, jeżeli ustalimy $n$, $p$ i $d$ to z CTG możemy znaleźć to prawdopodobieństwo. Ale tak naprawdę jeżeli ustalimy dowolne trzy z czterech wartości jakie tutaj mamy, to możemy wyznaczyć brakującą bez trudu. Wyobraźmy sobie, że chcemy mieć pewną ustaloną pewność, że trafimy do przedziału o ustalonym odchyleniu i chcę ustalić ile minimalnie $n$ potrzebuję. Na ustalenie pewności możemy spojrzeć w następujący sposób: możemy powiedzieć że trafimy do szukanego przedziału i pomylimy się z prawdopodobieństwem $\alpha$. Wtedy otrzymujemy równanie $$ P( np-d \leq S_n \leq np+d) = 1-\alpha$$. Po prostych przekształceniach i skorzystaniu z CTG (proszę samemu przeprowadzić przekształcenia) otrzymujemy $$ 2\Phi (\frac{d}{\sqrt{np(1-p)}}) -1 = 1-\alpha $$, co daje nam $$ n=\frac{d^2}{(\Phi^{-1}(\frac{2-\alpha}{2}))^2p(1-p)} $$ (do znalezienia funkcji odwrotnej do dystrybuanty korzystamy z $norm.ppf$). To oszacowanie będzie najgorsze dla $p=\frac{1}{2}$ więc jeżeli mamy jakąkolwiek możliwość oszacować $p$ należy z niej skorzystać. Ponieważ rozmiar próby powinien być liczbą całkowitą jako odpowiedź bierzemy sufit z otrzymanego wyniku.

In [7]:
def rozmiar(p,d,pb):#dod
  #p ograniczenie na prawdopodobieństwo uzyskania sukcesu, d-rozmiar odchylenia, pb - prawdopodobieństwo z jakim uzyskamy błąd
  ppf_arg=(2-pb)/2
  f1=norm.ppf(ppf_arg)
  expression=ceil(d**2/(f1**2*p*(1-p)))
  return expression #minimalny rozmiar próby
#przykład zastosowania rozmiar(0.5,3.375,0.5) daje wynik 101